In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.regularizers import l2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import optimizers
from keras import backend as K

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
md = pd.read_csv("musk_csv.csv" , index_col= None )

In [0]:
md = md.drop(['ID','conformation_name'] , axis =1)

In [0]:
# integer encode
label_encoder = LabelEncoder()
integer_encoded_t = label_encoder.fit_transform(md['molecule_name'])
print(integer_encoded_t)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded_t = integer_encoded_t.reshape(len(integer_encoded_t), 1)
onehot_encoded_t = onehot_encoder.fit_transform(integer_encoded_t)

In [0]:
ohe = pd.DataFrame(onehot_encoded_t)
train_d = pd.concat([ohe, md], axis = 1)
train_d = train_d.drop('molecule_name', axis = 1)
train_d

In [0]:
X_tran = train_d.drop('class', axis = 1)
X_tran

In [0]:
y_train = train_d['class']


In [0]:
X_train_n = tf.keras.utils.normalize(X_tran,axis=1,order=2)
X_train_n

In [0]:
model = Sequential()
model.add(Dense(134, input_dim=268, activation='relu'))
model.add(Dense(62, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [0]:

keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)
model.compile(loss='binary_crossentropy', optimizer= 'RMSprop' , metrics=['accuracy'])
# Fit the model
hist = model.fit(X_train_n, y_train, validation_split=0.2, epochs=8, batch_size= 8)

In [0]:
# Plot training & validation accuracy values
plt.plot(hist.history['acc'])
plt.plot(hist.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Cross'], loc='upper left')
plt.show()

In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
loss, accuracy, f1_score, precision, recall = model.evaluate(, verbose=0)